In [6]:
import numpy as np
import pandas as pd

from pyomo import environ as omo
from sfa.optimize.base import OptimizerBase

%matplotlib inline
from IPython import display
from IPython.core.pylabtools import figsize
from seaborn import plt

In [87]:
tables = pd.DataFrame(
    [["one", 5, 1.0],
     ["two", 5, 0.5],
     ["three", 5, 0.25]],
    columns=["name", "capacity", "desirability"])

guests = pd.DataFrame(
    [["Steve", ["Nancy"], "Jeff Family", "60+", "guy"],
     ["Nancy", ["Steve"], "Jeff Family", "60+", "girl"],
     ["Erica", [], "Jeff Family", "30s", "girl"],
     ["Gary", ["Jann"], "Beth Family", "60+", "guy"],
     ["Jann", ["Gary"], "Beth Family", "60+", "girl"],
     ["John", ["Whatsername"], "Beth Family", "20s", "guy"],
     ["Whatsername", ["John"], "Misc. +1s", "20s", "girl"],
     ["Matt", ["Steph"], "Beth Family", "30s", "guy"],
     ["Steph", ["Matt"], "Beth Family", "30s", "girl"],
     ["Ben", ["Annie"], "Jeff College", "20s", "guy"],
     ["Annie", ["Ben"], "Jeff College", "20s", "girl"],
     ["Max", ["Jenny"], "Jeff College", "20s", "guy"],
     ["Jenny", ["Max"], "Misc +1s", "20s", "girl"],
     ["Peter", ["Sav"], "Jeff Cali", "20s", "guy"],
     ["Sav", ["Peter"], "Jeff Cali", "20s", "girl"]
    ],
    columns=["name", "party", "cohort", "age", "gender"])

# Groups are assumed to have an affinity of +1 with themselves unless specified otherwise
affinities = pd.DataFrame(
    [["Jeff Family", "Jeff Family", 0],
     ["Jeff Family", "Beth Family", 1],
     ["Jeff Cali", "Jeff College", 0.5],
     ["20s", "30s", 0.5],
     ["20s", "60+", -1],
     ["30s", "60+", -0.5],
     ["girl", "girl", 0.1]
    ],
    columns=["x", "y", "affinity"])

In [167]:
def one_seat_per_guest(model, guest):
    assigned_seats = [asg for asg in model.assignments]

class SeatingModel(OptimizerBase):
    
    ADJACENT_CONNECTIVITY = 0.5
    TABLE_CONNECTIVITY = 1.0
    PARTY_STRENGTH = 1000.0
    
    def __init__(self, tables, guests, affinities):
        self.class_name = "SeatingModel"
        self._tables = tables
        self._guests = guests
        self._affinities = affinities

        self._table_capacities = dict(zip(
                self._tables.name, self._tables.capacity))
        self._seats = []
        self._connectivity = {}
        self._pair_aff_dict = {}
        
        self._add_seats()
        self._add_connectivity()
        self._add_pair_affinities()
        
        self.model = None
        
    def _add_seats(self):
        self._seats = sum(
            [[(n, x) for x in np.arange(c)]
             for n, c in zip(self._tables.name, self._tables.capacity)],
            [])
        
    def _add_connectivity(self):
        self._connectivity = {}
        for x_table, x_seat in self._seats:
            xname = "{}-{}".format(x_table, x_seat)
            cap = self._table_capacities[x_table]
            for y_table, y_seat in self._seats:
                if (y_table != x_table) or (x_seat >= y_seat):
                    continue
                yname = "{}-{}".format(y_table, y_seat)
                adjacent = (
                    (x_seat == ((y_seat + 1) % cap)) or
                    (x_seat == ((y_seat - 1) % cap)))
                if adjacent:
                    self._connectivity[(xname, yname)] = self.ADJACENT_CONNECTIVITY
                else:
                    self._connectivity[(xname, yname)] = self.TABLE_CONNECTIVITY
                    
    def _add_pair_affinities(self):
        groups = set()
        group_cols = set(self._guests.columns) - set(["name", "party"])
        for col in group_cols:
            for group in self._guests[col].unique():
                groups.add(group)

        group_aff_dict = {}
        for group in groups:
            group_aff_dict[group, group] = 1.0
        for _, row in self._affinities.iterrows():
            gpair = tuple(sorted((row.x, row.y)))
            group_aff_dict[gpair] = float(row.affinity)
        
        self._pair_aff_dict = {}
        for _, x in self._guests.iterrows():
            x_groups = [x[col] for col in group_cols] + [x["name"]]
            for _, y in self._guests.iterrows():
                if x["name"] >= y["name"]:
                    continue
                y_groups = [y[col] for col in group_cols] + [y["name"]]
                affinity = 0.0
                if x["name"] in y.party or y["name"] in x.party:
                    affinity += self.PARTY_STRENGTH
                for x_group in x_groups:
                    for y_group in y_groups:
                        gpair = tuple(sorted((x_group, y_group)))
                        affinity += group_aff_dict.get(gpair, 0.0)
                self._pair_aff_dict[(x["name"], y["name"])] = affinity

    def _add_indices(self):
        model = self.model
        model.guests = omo.Set(
            initialize=self._guests.name)
        model.seats = omo.Set(
            initialize=["{}-{}".format(n, x) for n, x in self._seats])
        model.pairs = omo.Set(
            initialize=self._pair_aff_dict.keys(),
            within=model.guests * model.guests)
        model.seatings = omo.Set(
            initialize=model.guests * model.seats)
        model.links = omo.Set(
            initialize=self._connectivity.keys(),
            within=model.seats * model.seats)

    def _add_parameters(self):
        model = self.model
        model.connectivity = omo.Param(
            model.links,
            initialize=self._connectivity,
            within=omo.NonNegativeReals)
        model.affinities = omo.Param(
            model.pairs,
            initialize=self._pair_aff_dict,
            default=0)

    def _add_variables_and_objective(self):
        """Adds optimizeable variables and objective to the model.
        
        variables
        assignments (bool indexed by seatings)
        
        constraints
        sum of assignments across each guest = 1
        sum of assignments across each seat = 1
        
        objective
        total_reward = 0
        For s1, s2 in links:
            str = conn_str[s1, s2]
            reward = 0
            for g1, g2 in pairs:
                p_aff = affinities[g1, g2]
                seated = (assignments[g1, s1] * assignments[g2, s2]) + (assignments[g2, s1] * assignments[g1, s2])
                reward += p_aff * seated
            total_reward += str * reward

        vars = tables * (cap + ((cap^2) / 2) * n_guest^2
        """
        model = self.model
        model.assignments = omo.Var(
            model.seatings,
            domain=omo.Binary)
        
        # Accumulate rewards
        rewards = []
        for s1, s2 in model.links:
            con = model.connectivity[(s1, s2)]
            for g1, g2 in model.pairs:
                if g1 >= g2:
                    continue
                aff = model.affinities[(g1, g2)]
                paired = (
                    (model.assignments[g1, s2] * model.assignments[g2, s2]) +
                    (model.assignments[g2, s2] * model.assignments[g1, s2]))
                rewards.append(con * aff * paired)
        rewards = [r for r in rewards if r is not 0]
        self._rewards = rewards
                
        # Objective
        model.objective = omo.Objective(
            expr=sum(rewards),
            rule=omo.maximize)

    def _add_constraints(self):
        """Adds constraints to the model."""
        model = self.model
        model.fully_seated = omo.Constraint(
            model.guests,
            rule=one_seat_per_guest)

In [168]:
foo = SeatingModel(tables, guests, affinities)

In [169]:
foo._build_model()

ERROR: Rule failed when generating expression for objective objective:
	TypeError: 'int' object is not callable
ERROR: Constructing component 'objective' from data=None failed:
	TypeError: 'int' object is not callable


TypeError: 'int' object is not callable

In [170]:
sum([r is 0 for r in foo._rewards])

0

#### indices
- seats
- guests
- pairs (within guests * guests)
- seatings (within guests * seats)
- links (within seats * seats)

#### parameters
- connection strengths (float indexed by links)
- affinities (float indexed by pairs)

#### variables
- assignments (bool indexed by seatings) 

#### constraints
- sum of assignments across each guest = 1
- sum of assignments across each seat = 1

#### objective
```
total_reward = 0
For s1, s2 in links:
    str = conn_str[s1, s2]
    reward = 0
    for g1, g2 in pairs:
        p_aff = affinities[g1, g2]
        seated = (assignments[g1, s1] * assignments[g2, s2]) + (assignments[g2, s1] * assignments[g1, s2])
        reward += p_aff * seated
    total_reward += str * reward

vars = tables * (cap + ((cap^2) / 2) * n_guest^2
```



In [119]:
omo.RealInterval?